# Ataque cíclico ao RSA

Ricardo Silva a71532

Francisca Fernandes a72450

João Cerqueira a65432

José Barbosa a69136

Descreva e exemplifique o ataque cíclico ao RSA.

**NOTA**: TP2 - 2ª aula gravada, a partir do minuto 43 está explicado o ataque.

Para testar este ataque vamos definir uma chave pública e uma chave privada usando o mesmo algoritmo que definimos no primeiro trabalho.

In [13]:
def RSA_Key(nbits = 512):
    p = random_prime(2^(nbits//2), 2^(nbits//2-1))
    q = random_prime(2^(nbits//2+1), 2^(nbits//2))
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    while gcd(e, m) != 1:
        e = randint(2, m-1)
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    PrivKey = d
    return PubKey, PrivKey

Geramos então as chaves e guardamos os valores _e_ e *n* da chave pública.

In [14]:
ChPub, ChPriv = RSA_Key(16)
n = ChPub[0] # n = p*q
e = ChPub[1]
print("Chave publica:", ChPub, "\n")
print("Chave privada:", ChPriv)

Chave publica: (42173, 23891) 

Chave privada: 10811


O remetente gera então a cifra da mensagem e envia para o destinatário.

In [15]:
msg = 1234
C = power_mod(msg, e, n)
C

39076

No entanto nós capturamos a cifra e temos os valores da chave pública.

Temos a chave pública $(n,e)$ e a mensagem cifrada $C$. Queremos decifrar a mensagem cifrada $C$ de forma a obter a mensagem original $P$. Para isso, vamos construir a sucessão {$C_{j}$}, com $1\leq C_{j} < n$, em que $C_{1} = C^{e}\ mod\ n$, $C_{2} = C_{1}^{e}\ mod\ n$, ..., $C_{j+1} = C{j}^{e}\ mod\ n$. Se encontrarmos $C_{j} = C$, então sabemos que $C_{j-1} = P$, e temos então a nossa mensagem decifrada.

Interceptamos a mensagem cifrada $C$, e sabemos que $C = m^e\ mod\ n$.
Começamos então por definir $C_{1}$:

In [16]:
C1 = power_mod(C, e, n)
C1

40227

$C_{1} \neq C$, logo temos de continuar a calcular $C_{2}$, $C_{3}$, ..., $C_{j}$ até encontrarmos um valor que iguale C.
Podemos então definir um ciclo que calcula $C_{j} = C_{j-1}^e\ mod\ n$  até que encontre o valor de j que nos permita decifrar a mensagem.

In [34]:
Cj = C1
i = 1

while Cj != C:
    i = i + 1
    Cj = power_mod(Cj,e,n)

print("Cj =", Cj, ", i =", i)

Cj = 39076 , i = 42


Podemos então definir os seguintes algoritmos que nos permitem descobrir o valor de j e a mensagem original  P a partir da chave pública _(n,e)_ e da mensagem cifrada C.

In [31]:
# Este algoritmo indica qual o valor de j de quebra a cifra
def RSACycleAttack_Find_j(n, e, C):
    i = 1
    Cj = power_mod(C, e, n)
    while Cj != C:
        i = i + 1
        Cj = power_mod(Cj, e, n)
    return i

RSACycleAttack_Find_j(n, e, C)

42

In [32]:
# Este algoritmo devolve a mensagem decifrada
def RSACycleAttack(n, e, C):
    i = 1
    C_aux = C
    Cj = power_mod(C, e, n)
    while Cj != C:
        i = i + 1
        C_aux = Cj
        Cj = power_mod(Cj, e, n)
    return C_aux

RSACycleAttack(n,e,C)

1234

# Ataque 1

### Factorizar _n_ sabendo $\varphi(n)$

Suponhamos _n = pq_. Sabendo $\varphi(n)$, é fácil computar *p* e _q_. Temos que:
$\varphi(n)\ =\ (p-1)(q-1)\ =\ pq\ -\ (p+q)\ +\ 1$,

então concluimos que 

$pq = n$ e $p+q=n+1-\varphi(n)$.

Logo, conhecemos o polinómio

$x^2-(p+q)x+pq\ =\ (x-p)(x-q)$

cujas raízes são _p_ e *q*. Estas raízes podem ser cálculadas usando a fórmula quadrática.



In [33]:
n

500069593

In [34]:
phi = euler_phi(n)
phi

499940788

In [35]:
# f = x² - (n+1-phi)*x+n
b = -(n+1-phi)
b

-128806

In [36]:
n

500069593

In [37]:
# f = x² - b * x + n
a = 1
c = n

# resultado da fórmula quadrática
p = (-b+sqrt(b^2-4*a*c))/(2*a)
p

124799

In [38]:
q = n/p
q

4007

In [39]:
n == p*q

True

Uma vez que já conseguimos obter n, então conseguimos calcular a chave privada d:

In [40]:
d = power_mod(e, -1, phi)
d

373583859

In [41]:
d == ChPriv

True

# Ataque 2

In [58]:
# Num ataque cíclico sabemos a chave pública (e, n) e interceptamos a
# mensagem cifrada. Queremos descobrir a mensagem original.
# Se conseguirmos computar a factorização n = p*q, então conseguimos
# calcular phi(n) = (p-1)(q-1) e logo somos capazes de obter d.

# se 'p' e 'q' forem muito próximos um do outro e p>q, então podemos
# concluir que:
#s = (p+q)/2 # será um valor  pequeno

#t = (p+q)/2 # é pouco maior que sqrt(n) e t²-n=s² é um quadrado perfeito.

# Então tentamos:
t = ceil(sqrt(n))
t
#[for i in range(10000)]

22363

In [59]:
flag = False
s = -1
while flag == False:
    if(sqrt(t^2-n).is_integer()):
        print("s =", sqrt(t^2-n),", t =", t)
        s = sqrt(t^2-n)
        flag = True
    else:    
        t = t + 1

s = 60396 , t = 64403


In [60]:
s, t

(60396, 64403)

In [61]:
p2 = t + s
p2

124799

In [62]:
q2 = t - s
q2

4007

In [63]:
p == p2 and q == q2

True